<a href="https://colab.research.google.com/github/techpvk/EVR1/blob/main/s6/S6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [92]:
#Approch1
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1,stride=2)
        self.conv2 = nn.Conv2d(16, 16, 3, padding=0)
        self.conv2_bn = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.do1 = nn.Dropout(0.25)
        self.conv3 = nn.Conv2d(16, 32, 3)
        self.conv4 = nn.Conv2d(32, 32, 3)
        self.conv4_bn = nn.BatchNorm2d(32)
        self.do2 = nn.Dropout(0.25)
        self.conv6 = nn.Conv2d(64, 64, 3)
        self.fc1 = nn.Linear(32, 10)
    def forward(self, x):
        x = self.pool1(self.conv2_bn(F.relu(self.conv2(F.relu(self.conv1(x))))))
        x = self.do1(x)
        x = F.adaptive_avg_pool2d(self.conv4_bn(F.relu(self.conv4(F.relu(self.conv3(x))))),(1,1))
        x = self.do2(x)
        x = x.view(-1, 32)
        x = F.relu(self.fc1(x))
        return F.log_softmax(x)

In [93]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 14, 14]             160
            Conv2d-2           [-1, 16, 12, 12]           2,320
       BatchNorm2d-3           [-1, 16, 12, 12]              32
         MaxPool2d-4             [-1, 16, 6, 6]               0
           Dropout-5             [-1, 16, 6, 6]               0
            Conv2d-6             [-1, 32, 4, 4]           4,640
            Conv2d-7             [-1, 32, 2, 2]           9,248
       BatchNorm2d-8             [-1, 32, 2, 2]              64
           Dropout-9             [-1, 32, 1, 1]               0
           Linear-10                   [-1, 10]             330
Total params: 16,794
Trainable params: 16,794
Non-trainable params: 0
--------------------------------------------------------------

<ipython-input-92-dfd44457f8b9>:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [88]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [89]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [90]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-86-7a88828dff5c>:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.12461546808481216 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.85it/s]



Test set: Average loss: 0.0580, Accuracy: 9840/10000 (98%)



loss=0.1318528801202774 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.32it/s]



Test set: Average loss: 0.0402, Accuracy: 9876/10000 (99%)



loss=0.04050195962190628 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 26.53it/s]


...


Test set: Average loss: 0.0228, Accuracy: 9923/10000 (99%)



loss=0.03948802873492241 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.00it/s]



Test set: Average loss: 0.0210, Accuracy: 9930/10000 (99%)



loss=0.02534470520913601 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 26.16it/s]



Test set: Average loss: 0.0214, Accuracy: 9929/10000 (99%)

